In [ ]:
!pip3 install xgboost

In [ ]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19878 sha256=d23749f79e002adef9df421ebaf5e1c7cd7e76077d089d7eaff962233f45abe9
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, StackingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
from sklearn.metrics import mean_absolute_error
#from vecstack import stacking

In [ ]:
train = pd.read_csv('train_final.csv')
#pretest = pd.read_csv('pretest_impute_linear_with_smiles.csv')
pretest = pd.read_csv('test_28_final.csv')

In [ ]:
#train_x = train.drop(columns=['CO2_working_capacity [mL/g]','functional_groups','MOFname','smiles'])
#train_y = train['CO2_working_capacity [mL/g]']

train_x = train.drop(columns=['gsa','vsa','CO2_working_capacity [mL/g]','functional_groups','MOFname','smiles','MolecularFormula','CID','InChI','InChIKey','IUPACName'])
train_y = train['CO2_working_capacity [mL/g]']

In [ ]:
#test_x = pretest.drop(columns=['MOFname'])
#train_x = pd.get_dummies(train_x)
X = pd.get_dummies(train_x)
train_x_column = train_x.columns
#X, Xval, y, yval = train_test_split(train_x,train_y, test_size =0.2)


In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X) 
y= train_y
#Xval = sc.transform (Xval)

In [ ]:
for col in ['metal_linker', 'organic_linker1', 'organic_linker2']:
    pretest[col] = pretest[col].astype('category')
#test= test.drop(columns=['MOFname','functional_groups','smiles','MolecularFormula','InChI','InChIKey','IUPACName'])
test= pretest.drop(columns=['functional_groups','MOFname','smiles','MolecularFormula','CID','InChI','InChIKey','IUPACName'])

In [ ]:
pretest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 49 columns):
 #   Column                                         Non-Null Count  Dtype   
---  ------                                         --------------  -----   
 0   MOFname                                        17000 non-null  object  
 1   volume [A^3]                                   17000 non-null  float64 
 2   weight [u]                                     17000 non-null  float64 
 3   density [g/cm^3]                               17000 non-null  float64 
 4   surface_area [m^2/g]                           17000 non-null  float64 
 5   smiles                                         17000 non-null  object  
 6   MolWt                                          17000 non-null  float64 
 7   MolLogP                                        17000 non-null  float64 
 8   NumValenceElectrons                            17000 non-null  int64   
 9   HeavyAtomCount                         

In [ ]:
test = pd.get_dummies(test)
test_column = test.columns

In [ ]:
l = []
for col in train_x_column:
    if col not in test_column:
        l.append(col)
listofzeros = [0] *17000
for col in l:
    test[col]= listofzeros
test.shape

(17000, 171)

In [ ]:
#for col in ['metal_linker', 'organic_linker1', 'organic_linker2']:
    #pretest[col] = pretest[col].astype('category')
#test_x= pretest.drop(columns=['MOFname'])
#test_x = pd.get_dummies(test_x)
#Xtest_column = test_x.columns
l = []
for col in train_x_column:
    if col not in Xtest_column:
        l.append(col)
listofzeros = [0] *17000
for col in l:
    test_x[col]= listofzeros
test_x.shape

(17000, 1625)

In [ ]:
test_x = test.reindex(columns = train_x_column)

In [ ]:
test_x = sc.transform (test)

ValueError: ignored

In [ ]:
test_x.shape

(17000, 41)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 139)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 139,
 'verbose': 0,
 'warm_start': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 3]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [10, 35, 60, 85, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 3],
 'min_samples_split': [2, 5],
 'n_estimators': [10, 57, 105, 152, 200]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=139, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, y)  

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 47.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 99.6min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [ ]:
rf_random.best_params_

NameError: ignored

In [ ]:
from sklearn.ensemble import RandomForestRegressor
#ใช้ความลึกของต้นไม้=10
max110_forest_model = RandomForestRegressor(max_depth=110,random_state=139,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=200)
max110_forest_model.fit(X, y)
predict_max110_forest = max110_forest_model.predict(Xval)
mae_max110_forest = mean_absolute_error(y_true=yval,y_pred=predict_max110_forest)
math.log10(mae_max110_forest)

1.3336064569433672

In [ ]:
pred1 = model.predict(test_x)

In [ ]:
predict

array([-1.70557089,  0.60120879, -2.4607961 , ..., -5.89603907,
       -7.13064657, -7.4190353 ])

In [ ]:
df = pd.DataFrame()
df['id'] = pretest['MOFname']
df['CO2_working_capacity [mL/g]'] = predict
df

,id,CO2_working_capacity [mL/g]
0,mof_unit_68614,-1.705571
1,mof_unit_68615,0.601209
2,mof_unit_68616,-2.460796
3,mof_unit_68617,-5.402625
4,mof_unit_68618,-4.634194
...,...,...
16995,mof_unit_85609,-9.478695
16996,mof_unit_85610,-7.130647
16997,mof_unit_85611,-5.896039
16998,mof_unit_85612,-7.130647


In [ ]:
df['id'] = df['id'].apply(lambda x:x.replace("mof_unit_",""))

In [ ]:
df.to_csv('submission_phase2_adadt2.csv',index=False)

In [ ]:
max1102_forest_model = RandomForestRegressor(max_depth=110,random_state=139,min_samples_leaf=1,max_features='sqrt',min_samples_split=2,n_estimators=300)
max1102_forest_model.fit(X, y)
predict_max1102_forest = max1102_forest_model.predict(Xval)
mae_max1102_forest = mean_absolute_error(y_true=yval,y_pred=predict_max1102_forest)
math.log10(mae_max1102_forest)

1.3326887556512548

In [ ]:
max1103_forest_model = RandomForestRegressor(bootstrap=False,max_depth=150,random_state=139,min_samples_leaf=1,max_features='sqrt',min_samples_split=2,n_estimators=300)
max1103_forest_model.fit(X, y)
predict_max1103_forest = max1103_forest_model.predict(Xval)
mae_max1103_forest = mean_absolute_error(y_true=yval,y_pred=predict_max1103_forest)
math.log10(mae_max1103_forest)

1.326168499425239

In [ ]:
pred_max1103_forest = max1103_forest_model.predict(test_x)

In [ ]:
ax1104_forest_model = RandomForestRegressor(bootstrap=False,max_depth=200,random_state=139,min_samples_leaf=1,max_features='sqrt',min_samples_split=2,n_estimators=1000)
max1104_forest_model.fit(X, y)
predict_max1104_forest = max1104_forest_model.predict(Xval)
mae_max1104_forest = mean_absolute_error(y_true=yval,y_pred=predict_max1104_forest)
math.log10(mae_max1104_forest)

1.3234358808790676

In [ ]:
pred_max1104_forest = max1104_forest_model.predict(test_x)

In [ ]:
max1105_forest_model = RandomForestRegressor(bootstrap=False,max_depth=300,random_state=139,min_samples_leaf=2,max_features='auto',min_samples_split=4,n_estimators=600)
max1105_forest_model.fit(X, y)
predict_max1105_forest = max1105_forest_model.predict(Xval)
mae_max1105_forest = mean_absolute_error(y_true=yval,y_pred=predict_max1105_forest)
math.log10(mae_max1105_forest)

1.4809327551092233

In [ ]:
max1103_forest_model

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
model = BaggingRegressor(base_estimator = RandomForestRegressor(), n_estimators = 150, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

In [ ]:
pred_max1105_forest = max1105_forest_model.predict(test_x)

In [ ]:
train2 = pd.read_csv('train_impute_linear_with_smiles2.csv')


In [ ]:
train_x2 = train2[['fact1','fact2','fact3','fact4']]

In [ ]:
X2, Xval2, y2, yval2 = train_test_split(train_x2,train_y, test_size =0.2)

In [ ]:
max110_forest_model_pca = RandomForestRegressor(bootstrap=False,max_depth=200,random_state=139,min_samples_leaf=1,max_features='sqrt',min_samples_split=2,n_estimators=300)
max110_forest_model_pca.fit(X2, y2)
predict_max110_forest_pca = max110_forest_model_pca.predict(Xval2)
mae_max110_forest_pca = mean_absolute_error(y_true=yval2,y_pred=predict_max110_forest_pca)
math.log10(mae_max110_forest_pca)

1.5817368769857483

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor

regr = BaggingRegressor(base_estimator=SVR(),
                         n_estimators=10, random_state=139)

regr.fit(X, y)
predict_bag = regr.predict(Xval)
mae_bag = mean_absolute_error(y_true=yval,y_pred=predict_bag)
math.log10(mae_bag)

NameError: ignored

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.2,0.5],'subsample' : [0.5, 0.2],
            'n_estimators' : [100,300],
            'max_depth'    : [20,100]}
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
grid_GBR.fit(X, y)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.2, loss='ls', max_depth=20,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=0.5, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

 The best score across ALL searched params:
 0.8369842707852319

 The best parameters across ALL searched params:
 {'learning_rate': 0.2, 'max_depth': 20, 'n_estimators': 100, 'subsample': 0.5}


In [ ]:
from xgboost import XGBRegressor
xg = XGBRegressor(random_state=139)
xg.fit(X, y)
pred_xgb = xg.predict(Xval)
mae_xgb = mean_absolute_error(y_true=yval,y_pred=pred_xgb)
math.log10(mae_xgb)

ValueError: ignored

In [ ]:
!pip install lazypredict

In [ ]:
import lazypredict

In [ ]:
from lazypredict.Supervised import LazyRegressor
from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)


In [ ]:
reg.fit(X, Xval, y, yval)


ImportError: ignored

In [ ]:
ImportError: cannot import name 'values_from_object' from 'pandas._libs.lib' #337

SyntaxError: ignored

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True,False],
    'max_depth': [150,300],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [3, 5],
    'n_estimators': [200, 500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
# Fit the grid search to the data
grid_search.fit(X, y)
grid_search.best_params_

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 87.2min


In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model = BaggingRegressor(base_estimator = RandomForestRegressor(), n_estimators = 40, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3374657816882602

In [ ]:
model = BaggingRegressor(base_estimator = DecisionTreeRegressor(), n_estimators = 50, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3430598958206157

In [ ]:
model = BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth=200,max_features='sqrt'), n_estimators = 100, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3340539710536616

In [ ]:
model = BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth=200), n_estimators = 200, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3411243329905154

In [ ]:
model = BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth=300,max_features='sqrt'), n_estimators = 100, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3340539710536616

In [ ]:
model = BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth=400,max_features='sqrt'), n_estimators = 300, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.332114437608023

In [ ]:
model = BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth=400,max_features='sqrt'), n_estimators = 400, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3319240296398474

In [ ]:
pred = model1.predict(test_x)

In [ ]:
test_copy = pretest.copy()

In [ ]:
df = pd.DataFrame()
df['id'] = test_copy['MOFname']
df['CO2_working_capacity [mL/g]'] = pred

In [ ]:
df['id'] = df['id'].apply(lambda x:x.replace("mof_unit_",""))

In [ ]:
df.to_csv('submission_phase2_abrf_25.csv',index=False)

In [ ]:
df

,id,CO2_working_capacity [mL/g]
0,68614,193.290527
1,68615,57.106033
2,68616,69.793561
3,68617,62.814677
4,68618,71.881857
...,...,...
16995,85609,-6.898100
16996,85610,1.281846
16997,85611,0.831274
16998,85612,-1.004605


In [ ]:
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor(base_estimator = RandomForestRegressor(), random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=300),n_estimators = 200, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3270025053105674

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=300),n_estimators = 200, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3173768358855653

In [ ]:
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=300),n_estimators = 400, random_state = 139,learning_rate=0.6,loss='exponential')
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

KeyboardInterrupt: ignored

In [ ]:
model = BaggingRegressor(base_estimator = RandomForestRegressor(), n_estimators = 20, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

NameError: ignored

In [ ]:
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3276770734902623

In [ ]:
model = BaggingRegressor(base_estimator = XGBRegressor(max_depth=100), n_estimators = 20, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

[17:05:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:08:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:12:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:15:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:17:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:20:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:22:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

1.323804995747967

In [ ]:
from xgboost import XGBRegressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor(base_estimator = RandomForestRegressor(),n_estimators = 15, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3176316180776044

In [ ]:
model = AdaBoostRegressor(base_estimator = RandomForestRegressor(),n_estimators = 20, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.32644588152688

In [ ]:
model1 = AdaBoostRegressor(base_estimator = RandomForestRegressor(),n_estimators = 25, random_state = 139)
model1.fit(X,y)
predict1 = model1.predict(Xval)
mae_model1 = mean_absolute_error(y_true=yval,y_pred=predict1)
math.log10(mae_model1)

1.326696684204481

In [ ]:
model1 = AdaBoostRegressor(base_estimator = XGBRegressor(n_estimators=200,max_depth=200),n_estimators = 20, random_state = 139)
model1.fit(X,y)
predict1 = model1.predict(Xval)
mae_model1 = mean_absolute_error(y_true=yval,y_pred=predict1)
math.log10(mae_model1)

[10:11:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: ignored

In [ ]:
from xgboost import XGBRegressor

xg = XGBRegressor(n_estimators=200,max_depth=100,random_state=139)
# Add silent=True to avoid printing out updates with each cycle
xg.fit(X, y)

[10:00:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=100, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=139,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
predict = xg.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3373845433497553

In [ ]:
model = BaggingRegressor(base_estimator = RandomForestRegressor(), n_estimators = 20, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3282648469971832

In [ ]:
Ada = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=300),n_estimators = 200, random_state = 139)
Bagg = BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth=300), n_estimators = 200, random_state = 139)
Rforest = RandomForestRegressor(max_depth=20,random_state=139,n_estimators=300)

#---------Prediction------------------------------
#list of base models
level0 = list()
level0.append(('Ada', Ada))
level0.append(('Bagg', Bagg))
level0.append(('Rforest', Rforest))
# define meta learner model
level1 = LinearRegression()
# define the stacking model
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
model.fit(X, y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3232405187296485

In [ ]:
model1 = VotingRegressor(estimators=level0,)
model1.fit(X, y)
predict1 = model1.predict(Xval)
mae_model1 = mean_absolute_error(y_true=yval,y_pred=predict1)
math.log10(mae_model1)

1.3261286075157321

In [ ]:
Ada = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=300),n_estimators = 300, random_state = 139)
Bagg = BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth=300), n_estimators = 300, random_state = 139)
Rforest = RandomForestRegressor(max_depth=300,random_state=139,n_estimators=300)

#---------Prediction------------------------------
#list of base models
level0 = list()
level0.append(('Ada', Ada))
level0.append(('Bagg', Bagg))
level0.append(('Rforest', Rforest))
model1 = VotingRegressor(estimators=level0,weights=[0.4,0.4,0.2])
model1.fit(X, y)
predict1 = model1.predict(Xval)
mae_model1 = mean_absolute_error(y_true=yval,y_pred=predict1)
math.log10(mae_model1)

1.3251096230018442

In [ ]:
params = { 'max_depth': [3, 10, 20, 100, 200],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}
xgbr = xgb.XGBRegressor(seed = 20)
clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring='neg_mean_absolute_error',
                         n_iter=25,
                         verbose=1)
clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest RMAE: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[06:00:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[06:00:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:00:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:00:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:00:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:00:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:00:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:01:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:01:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed: 170.2min finished


[08:50:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best parameters: {'subsample': 0.7, 'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.01, 'colsample_bytree': 0.4, 'colsample_bylevel': 0.4}
Lowest RMAE:  4.460590584023796


In [ ]:
xg = XGBRegressor(n_estimators=500,max_depth=50,random_state=139,learning_rate=0.01,subsample=0.7,colsample_bytree=0.4,colsample_bylevel=0.4,seed=20)
# Add silent=True to avoid printing out updates with each cycle
xg.fit(X, y)
predict = xg.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

[16:39:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


1.3296327732310436

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=30),n_estimators = 700, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3238651872056721

In [ ]:
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=300),n_estimators = 200, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3254151673541914

In [ ]:
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=300),n_estimators = 500, random_state = 139)
model.fit(X,y)
predict = model.predict(test_x)


ValueError: ignored

In [ ]:
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(),n_estimators = 500, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3296049153255436

In [ ]:
param_dist = {
 'n_estimators': [50, 100],
 'learning_rate' : [0.01,0.05,0.1,0.3,1],
 'loss' : ['linear', 'square', 'exponential']
 }
ada = AdaBoostRegressor(DecisionTreeRegressor)
pre_gs_inst = RandomizedSearchCV(,
 param_distributions = param_dist,
 cv=3,
 n_iter = 10,
 n_jobs=-1)

pre_gs_inst.fit(X_train, y_train)

In [ ]:
pparam_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [100,300,500],
              "learning_rate" : [0.01,0.05,0.1,0.3,1],
              "loss" : ['linear', 'square', 'exponential']}


In [ ]:
param_grid = {"base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [100,300,500],
              "learning_rate" : [0.01,0.05,0.1,0.3,1],
              "loss" : ['linear', 'square', 'exponential']}

DTC = DecisionTreeRegressor(random_state = 139)

ABC = AdaBoostRegressor(base_estimator = DTC,random_state=139)
# run grid search
grid_search_ABC = RandomizedSearchCV(ABC, param_distributions=param_grid,cv=3,n_iter = 15,n_jobs=-1)
grid_search_ABC.fit(X, y)
print("Best parameters:", grid_search_ABC.best_params_)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best parameters: {'n_estimators': 500, 'loss': 'linear', 'learning_rate': 1, 'base_estimator__splitter': 'random'}


In [ ]:
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=300,max_features='auto'),n_estimators = 500, random_state = 139)
model.fit(X,y)
predict = model.predict(Xval)
mae_model = mean_absolute_error(y_true=yval,y_pred=predict)
math.log10(mae_model)

1.3296049153255436